<center><h1>7SSG2059 Geocomputation 2016/17</h1></center>

<h1><center>Practical 9: Merging and Joining Dataframes in Panadas</h1></center>

<p><center><i>James Millington, 17 November 2016</i></center>


## Overview

Up until this point we have been working with individual datasets. But what if we have data in two datasets that we want to work with together, for example to look for correlations between variables. This will be useful for example to examine relationshis between weather and air quality variables or between socio-economic classifications and amenity values in neighbourhoods. 

In this practical we will look at how to `merge` and `join` Pandas dataframes. There's also a `concatenate` function. Each of these functions are slightly different:
- `merge` enables us to [combine](http://pandas.pydata.org/pandas-docs/stable/merging.html) two dataframes based on a column that is common between them
- `join` is used to [combine](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) two dataframes when they share a common index (e.g. a DateTimeIndex)
- `concatenate` [combines](http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects) dataframes regardless of common attributes. 

Here we will look at how to use `merge` to combine the NS-Sec data with additional data for the LSOAs into a single dataframe, and we'll see how `join` is useful to combine weather and air quality data for the same measurement times into a single dataframe. We'll then look at how to analyse these combined dataframes using correlation and regression next week.  

## Merge

If we have a column in each of two DataFrames that contains the same identifier for the remaining data, we can use the common identifier column to define how the two DataFrames are joined together. For example, the NS-SeC data are for LSOAs (distinct geographical regions) - if any additional data we have is also for LSOAs, as long as we we have a common way of identifying the LSOAs in both DataFrames we can merge the Data Frames. 

### NS-Sec and Amenity Values

The additional data you can use in conjunction with the NS-SeC data are found in `LSOA_ValuesData_London.csv` on KEATS. There are a variety of additional factors that you are free to explore, and you can read about them in the `AdditionalDataOverview.pdf` document also on KEATS. Smith (2010) used similar data in their study which will also likely help you to think about possible analyses you might make for your final report (e.g. between house prices and socio-economic indicators of LSOAs). 

These data are for housing and other amenity values for LSOAs in London. Consequently, we'll also use only NS-SeC data for London from now on - LSOA NS-SeC data for London only can be found in `Data_NSSHRP_UNIT_URESPOP_London.csv` on KEATS.

The code below loads the two data files for London LSOAs into memory as pandas DataFrames, tidies up their column names and drops rows with missing data.

In [1]:
import pandas as pd

#read NS-SeC data
nsCN = ["CDU_ID","GEO_CODE","GEO_LABEL","F2084","F2085","F2094","F2102","F2107","F2114","F2119","F2127","F2133","F2136"]  
nsDF = pd.read_csv('Data_NSSHRP_UNIT_URESPOP_London.csv', header=0, skiprows=[1], usecols=nsCN)   #read csv with headers, skipping notes row and no data column 15
nsDF.columns = ["CDU_ID","GEO_CODE","GEO_LABEL","Total","Group1","Group2","Group3","Group4","Group5","Group6","Group7","Group8","NC"]  
nsDF = nsDF.dropna(axis = 0)  #drop rows with missing data

#read Additional Values Data
valCN = ["lsoa11cd","median_price","avg_distance_to_station","positive_area","moderate_area","negative_area"]
valDF = pd.read_csv('LSOA_ValuesData_London.csv', header=0, usecols = valCN)  
valDF = valDF.dropna(axis = 0)  #drop rows with missing data

FileNotFoundError: [Errno 2] File b'Data_NSSHRP_UNIT_URESPOP_London.csv' does not exist: b'Data_NSSHRP_UNIT_URESPOP_London.csv'

If we check these data we have just read into memory, we can see that the column in `valDF` named `lsoa11cd` uses the same labels for LSOAs as the `GEO_CODE` column in `nsDF`. Very handy!  

In [ ]:
#check what the common columns are
nsDF.head()
valDF.head()

By renaming `lsoa11cd` to `GEO_CODE` we can use it with the Pandas `merge` function:

In [ ]:
valDF.columns = ["GEO_CODE","MedPrice","MeanStationDist", "PosArea", "ModArea", "NegArea"]  #rename to 'GEO_CODE'!

The `merge` of `nsDF` and `valDF` is then done ‘on’ the `GEO_CODE` column found in each DataFrame. 

In [ ]:
#merge the two data frames 
nsvalDF = pd.merge(nsDF, valDF, on = 'GEO_CODE')

Each value in the `GEO_CODE` column in `nsDF` is matched with the same value in the `GEO_CODE` column in `valDF` and the rows those values are found in are combined. The figure below illustrates the process (combining `left` and `right` on `key`). Read more about merge [here](http://pandas.pydata.org/pandas-docs/stable/merging.html). 

![Illustration of the Pandas merge function](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key.png)

Check you understand how something similar has been done for our LSOA data, combining `nsDF` and `valDF` on `GEO_CODE`:

In [ ]:
#check output
nsvalDF.head()
nsvalDF.info()

Let's save these data for later - they may be useful for your final report!

In [ ]:
nsvalDF.to_csv("LondonLSOAData.csv")
nsvalDF.to_pickle("LondonLSOAData.pkl") 

## Join

The `merge` functions uses a common column (Series) in two dataframes to combine them. If the _index_ of two dataframes is common we could also use `merge` to combine on the index. However, we would need to pass more arguments to the `merge` function, and another function called `join` has been designed specifically to combine on dataframe indexes. 

Here, we'll join some air quality time-series data to our weather time-series data using a common `DateTimeIndex`. 

First, so we can join it later, we'll load (and check) the weather data from our previous data manipulation (week 7):

In [ ]:
metDF = pd.read_pickle("CleanedHeathrowData2016.pkl")
print metDF.info()
print metDF.tail()

### Air Quality Data

The additional data we'll use with the Heathrow Weather data are air quality data have been downloaded from the Air Quality England [website](http://www.airqualityengland.co.uk/) (AQE 2016) for the [Hounslow Hatton Cross site](http://www.airqualityengland.co.uk/site/latest?site_id=HS7) (site HS7). This site was chosen as it is near Heathrow Airport. 

Air pollution is an important aspect of the ongoing argument about the construction of the third runway at Heathrow (e.g. GLA 2012). In particular, although Nitrogen Dioxide (NO2) concentrations around Heathrow, are lower than in the centre of London, they are still often above recommended levels (e.g. Heathrow 2012). By looking at relationships between weather and air quality we may begin to better understand the drivers of pollution.

However, as the air quaity data have also been automatically collected, we'll need to do some cleaning and manipulation of those data before we can join them with the weather data. 

### Cleaning Air Quality Data

When reading the data (in the next code block) we will read only the first 10 columns of the data to a DataFrame named `aqDF`, accounting for the need to skip lines. Also note there is a footer in the data so we use the `skipfooter` argument, but this also means we need to add the `engine` argument (read more about this in the `read_csv` [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) online).

In [ ]:
aqDF = pd.read_csv('AirQuality_HattonCross_2016.csv', header=0, skiprows=3, usecols=range(0,10), skipfooter = 1, engine = 'python')


Next we'll rename the columns to something more intuitive:

In [ ]:
aqCN = ['Date', 'Time', 'PM10', 'PM10_su', 'NO', 'NO_su', 'NO2', 'NO2_su', 'NOx', 'NOx_su']
aqDF.columns = aqCN

Let's check the head and tail of the data to see what we've got:

In [ ]:
print aqDF.head()
print aqDF.tail() 

Notcie that some columns (the ones we named `_su`) are labels not actual data. We can quickly remove those using the python built-in function `del` (this is an alternative to using a loop to retain only `float64` data types, plus `Date` and `Time`, for example). 

In [ ]:
del aqDF['PM10_su'] 
del aqDF['NO_su']   
del aqDF['NO2_su'] 
del aqDF['NOx_su'] 

print aqDF.tail() 

### Setting a Common DateTimeIndex

As the air quality data are a time series like the weather data, we'll create and use a matching `DateTimeIndex`. Later we'll be able to use this to join the air quality data to the weather data. 

However, before we can set `DateTimeIndex` we need to change how midnight is represented in the air quality data to match the MetOffice data; in the air quality data midnight is 24:00 but in the MetOffice data it is 00:00 (you can see by comparing the tails of `aqDF` and `metDF` we printed above). As discussed [here](http://stackoverflow.com/a/30536227), ideally midnight should be represented by 00:00 and should be the _first_ record in a given date (not the last).

To change the 24:00 to 00:00 we can use pandas `replace` method on the Time `column` of `aqDF`: 

In [ ]:
aqDF.Time.replace(to_replace = '24:00:00', value = '00:00:00', inplace = True) 

Now we can set start to the index like we have done before for the weather data (see week 7), but in a moment we'll note there is a problem with when midnight appears in the air quality data. So first we'll create a new temporary column named `DT` so we can keep track of different formats:

In [ ]:
aqDF["DT"] = pd.to_datetime(aqDF.Date + aqDF.Time, format='%d/%m/%Y%H:%M:%S')  #create a new series containing a datetime object
aqDF.index = aqDF["DT"]                                                        #set the index

aqDF.tail()

The problem here is that by changing 24:00 to 00:00 in the air quality data, dates are off by one day for the 00:00 timestamp - each day should _start_ at midnight, but now in the air quality each day _finishes_ with midnight. We can see this by comparing the first value for an individual date in each of our dataframes:

In [ ]:
print aqDF['2016-10-09'].index[0]
print metDF['2016-10-09'].index[0]

Days in `metDF` start at `00:00:00` (as we want) but days in `aqDF` start at `01:00:00`!

To resolve this we need to add one day to the values for 00:00:00 in `aqDF`. We can do this using the [functionality](https://docs.python.org/2/library/datetime.html#timedelta-objects) from the `datetime` library. We'll update the values in `DT` just so we can check what we're doing is right.

In [ ]:
import datetime as dt

oneday = dt.timedelta(days=1)                           #create a timedelta object of days = 1
aqDF.loc[aqDF['Time'] == '00:00:00','DT'] += oneday     #update the DT cell for rows with Time == 00:00 by adding oneday

aqDF.tail()

You can see from the last line of the tail of `aqDF` that whereas the original `Date` column _ended_ a given date at `00:00:00`, the `DT` column now _starts_ a date with `00:00:00`. So now we can use `DT` to set the `DateTimeIndex` again, this time correctly matching the weather data index.  

Furthermore, because we have set the `DateTimeIndex` to equal the `DT` column, the index has automatically updated to match!

We don't need the DT column anymore so let's drop that, along with any duplicate records that might exist:

In [ ]:
del aqDF['DT'] 
aqDF = aqDF.drop_duplicates()

Finally we need to re-index `aqDF` so that all hours are represented (even those that are missing - see week 7):

In [ ]:
aqfirstDate = aqDF.index[0]
aqlastDate = aqDF.index[len(aqDF.Date) - 1]
aqDF = aqDF.reindex(index=pd.date_range(start = aqfirstDate, end = aqlastDate, freq = '1H'), fill_value = None)   #http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reindex.html

print aqDF.info()

Let's quickly check that aqDF and metDF have consistent `DateTimeIndex`:

In [ ]:
print "Head of aqDF:\n", aqDF.head()
print "\nHead of metDF:\n", metDF.head()
print "\nTail of aqDF:\n", aqDF.tail() 
print "\nTail of metDF:\n", metDF.tail()

Looks good. Now we can move on to see how we can combine these various dataframes. 

### Now we're ready to join!

The [syntax](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) for the `join` function is quite straight-forward but can depend on the names of existing columns. For our current air quality and weather data frames we would `join` as follows: 

In [ ]:
aqmetDF = aqDF.join(metDF, lsuffix='_l', rsuffix='_r')
aqmetDF.info()

The above creates a new dataframe (`aqmetDF`) from `aqDF` and `metDF`. We need to specify `lsuffix` and `rsuffix` as we have `Date` and `Time` columns in both of our dataframes - the suffixes are be added to the original columns in the new dataframe created so that we don't have duplicate columns names.

If we didn't have duplicate column names wouldn't need suffixes, so let's keep only `float64` series for the weather data, dropping `WindGust` and `LocID` (as they are not as interesting as the other variables):  

In [ ]:
metDF = metDF.select_dtypes(include=['float64'])
del metDF['WindGust'] 
del metDF['LocID']

And we'll drop `Date` and `Time` from `aqDF`:

In [ ]:
del aqDF['Date'] 
del aqDF['Time']

Now that we have no duplicate column names in our dataframes (check using `.info()` if you like), our useage of `join` is much simpler:

In [ ]:
aqmetDF = aqDF.join(metDF)
aqmetDF.info()
aqmetDF.head()

Let's save this new combined dataframe to disk as it could also be useful for your final report. Before we do so, let's check if there's any values we want to drop from or change in the data using a pairplot to visualise:

In [ ]:
import seaborn as sb
%matplotlib inline 

sb.pairplot(aqmetDF.dropna(axis = 0))

Oh yes, we need to drop those 0 _Pressure_ values that don't make sense:

In [ ]:
aqmetDF.loc[aqmetDF.Pressure == 0] = None

What's going on PM10? Are the very high values erroneous do you think? I'll leave that for you to investigate... Let's save the data to disk for safe-keeping and future use:

In [ ]:
aqmetDF.to_pickle("HeathrowAQWeather2016.pkl")

## Summary

So we have now created two dataframes of combined data and saved these to disk:
- `LondonLSOAData.pkl`
- `HeathrowAQWeather2016.pkl`

You can load these data into a pandas dataframe easily using `read_pickle` and use them in your final reports.

## Exercises

Below are some exercises to help you think about the data you have created - feel free to work on all the exercises but you should be beginning to think about which data set you will focus on for your final report. We'll investigate in week 10 some more, but these exercises might get you started.

### NS-SeC and Values
House Price data are often heavily skewed with a long tail (many smaller values and very, very few large values).  

**Task 1.** Check the distribution of House Price data by creating plots using the seaborn distplot function. 

When working with House Price data we often used a transformed version of the data that is the logarithm of the original data. One reason is that by reducing the skew of the data we overcome some of the problems of heteroscedasticity but also linear regression models are will better fit more normally distributed data. 

**Task 2.** Create a new column in the nsvalDF DataFrame that contains the natural logarithm of the House Price _[Hint: use the numpy `log` function]_ Once you have created your new series (column) check the new distribution using a plot

Now you might compare the relationship between both the transformed and un-transformed house price Series and the NS-SeC Group Series to think about how each is related and which might be better for correlation and regression analyses. 

**Task 3.** Use a loop to create seaborn jointplots between tranformed and un-transformed house proce data with each of the SN-SeC Group populations

### Weather and Air Quality

It's always useful to thoroughly check your data to understand it sufficiently to interpret results properly. 

**Task 1.** Use a loop to plot time series for for all weather and air quality variables. When are the missing data? Are there any strange values?


**Task 2.** Run some other exploratory data analyses: jointplots and correlations for different combinations of variables. Next week we'll look at regression

### References
- AQE (2016) _Air Quality England_ [Online] Available at: http://www.airqualityengland.co.uk/ 
- GLA (2012) Air and noise pollution around a growing Heathrow Airport [Online] Available from: http://www.london.gov.uk/mayor-assembly/london-assembly/publications/tackling-air-and-noise-pollution-around-heathrow 
- Heathrow (2012) Heathrow Air Quality [Online] Available from: http://www.heathrow.com/file_source/Company/Static/PDF/Communityandenvironment/air-quality-strategy_LHR.pdf
- Smith, D. (2010) _Valuing housing and green spaces: Understanding local amenities, the built environment and house prices in London._ London: Greater London Authority. 